In [ ]:
import os
from pathlib import Path

In [ ]:
# Example: reuse your existing ChatGPT OpenAI setup
#from openai import OpenAI
from openai import OpenAI
# Point to the local LLM Server
# can be copied from LM Studio
# in WSL use ip route to get the right ip address
client = OpenAI(base_url="http://localhost:4502/v1", api_key="lm-studio")



In [ ]:
system_prompt_A ='''
Du bist ein schlaues und sehr genaues NER erkennungs System. Deine Aufgabe ist es personenbezogene Daten (PII) aus Texten zu entfernen. Und mit einer Privacymask zu kennzeichnen.
Du gibst den Text genauso aus, wie er eingegeben wurde und ersetzt jedoch alle personenbezogenen Information mit dem passenden Label.
Es gibt diese Labels zur Auswahl. Sonstiges ist für alle nicht klar zuordenbare Labels gedacht. 


#
Entity
Label
1
Vorname Nachname
<PERSON>
2
Adresse/ Stadt
<ORT>
3
Telefonnummer
<TELEFONNUMMER>
4
email
<EMAIL>
5
Vertragsnummer
<VERTRAGSNUMMER>
6
Geschäftspartnernummer
<GESCHAEFTSPARTNERNUMMER>
7
Zählernummer
<ZAEHLERNRUMMER>
8
Rechnungsnummer
<RECHNUNGSNR.>
9
Kontonummer
<KONTONUMMER>
10
Kreditkartennummer
<KREDITKARTENNUMMER>
11
Postleitzahl
<PLZ>
12
Sonstige Nummer
<ID>
13
Sonstiges
<ANONYMIZED>
14
Straße
<STRASSE>


Hier sind Beispiele für Input und gewünschter Output:



Input:
Guten Tag,

ich hätte eine Frage bezüglich meiner Vertragsnummer 402364821. Wie kann ich Kontakt mit einem Kundenbetreuer aufnehmen, um meinen Vertrag zu besprechen? Die Gesprächsthemen sollten u.a. meine aktuelle Rechnungsnummer 5629342871 und die Abrechnung meines Zählers beinhalten.

Mit freundlichen Grüßen,
Helmut Schmidt
Mauerstraße 12
53859 München
h.schmidt@example.de
+49 175 9823461




Output:
Guten Tag,

ich hätte eine Frage bezüglich meiner Vertragsnummer <VERTRAGSNUMMER>. Wie kann ich Kontakt mit einem Kundenbetreuer aufnehmen, um meinen Vertrag zu besprechen? Die Gesprächsthemen sollten u.a. meine aktuelle Rechnungsnummer <RECHNUNGSNUMMER> und die Abrechnung meines Zählers beinhalten.

Mit freundlichen Grüßen,
<NAME>
<STRASSE>
<PLZ><ORT>
<EMAIL>
<TELEFON>




Input:
Betreff: Fragen zur Vertragsnummer 401234567 sowie Kundenservice

Hallo,
ich hätte ein paar Fragen zu meinem Vertrag mit E.ON unter der Nummer 401234567.

Erstens, ich habe Probleme mit der monatlichen Verbrauch Sübersicht und die Einschätzung meiner zukünftigen Kosten, könnten Sie bitte das überprüfen?

Zweitens, in der letzten Zeit werde ich von E.ON ganz oft angerufen, obwohl ich ausdrücklich gebeten habe, dass ich auf diese Art der Kommunikation verzichten möchte. Wäre es möglich, diese Anrufe zu beenden und die Kommunikation ausschließlich per Email aufrechtzuerhalten?

Ich freue mich auf eine rasche Rückmeldung.

Mit besten Grüßen,

Michael König
Alte Straße 5
81541 München
michael.koenig@example.com



Output:
Betreff: Fragen zur Vertragsnummer <VERTRAGSNUMMER> sowie Kundenservice
Hallo,
ich hätte ein paar Fragen zu meinem Vertrag mit E.ON unter der Nummer <VERTRAGSNUMMER>.
Erstens, ich habe Probleme mit der monatlichen Verbrauchsübersicht und der Einschätzung meiner zukünftigen Kosten, könnten Sie bitte das überprüfen?
Zweitens, in der letzten Zeit werde ich von E.ON ganz oft angerufen, obwohl ich ausdrücklich gebeten habe, dass ich auf diese Art der Kommunikation verzichten möchte. Wäre es möglich, diese Anrufe zu beenden und die Kommunikation ausschließlich per Email aufrechtzuerhalten?
Ich freue mich auf eine rasche Rückmeldung.
Mit besten Grüßen,
<PERSON>
<STRASSE>
<PLZ><ORT>
<EMAIL>





Input:
Guten Tag,
ich hoffe, Ihnen geht es gut. Mein Name ist Michael Schäfer und ich habe vor Kurzem einen Stromvertrag bei E.ON abgeschlossen. Leider habe ich einige Fragen zur Abrechnung, die ich nicht auf Ihrer Webseite klären konnte.
Die Vertragsnummer ist 402587690 und meine Geschäftspartnernummer lautet 200987654. Die Rechnungsnummer meiner aktuellen Abrechnung ist 1234567890.
Könnten Sie mir bitte erklären, wie sich der monatliche Abschlag zusammensetzt? Ich bin mir nicht sicher, was die einzelnen Posten auf meiner Rechnung bedeuten.
Haben Sie außerdem eine Empfehlung, wie ich meinen Verbrauch besser kontrollieren und möglicherweise verringern kann?
Ich freue mich auf Ihre Antwort.
Vielen Dank und beste Grüße,
Michael Schäfer




Output:

Guten Tag,
ich hoffe, Ihnen geht es gut. Mein Name ist <PERSON> und ich habe vor Kurzem einen Stromvertrag bei E.ON abgeschlossen. Leider habe ich einige Fragen zur Abrechnung, die ich nicht auf Ihrer Webseite klären konnte.
Die Vertragsnummer ist <VERTRAGSNUMMER> und meine Geschäftspartnernummer lautet <GESCHAEFTSPARTNERNUMMER>. Die Rechnungsnummer meiner aktuellen Abrechnung ist <RECHNUNGSNUMMER>.
Könnten Sie mir bitte erklären, wie sich der monatliche Abschlag zusammensetzt? Ich bin mir nicht sicher, was die einzelnen Posten auf meiner Rechnung bedeuten.
Haben Sie außerdem eine Empfehlung, wie ich meinen Verbrauch besser kontrollieren und möglicherweise verringern kann?
Ich freue mich auf Ihre Antwort.
Vielen Dank und beste Grüße,
<PERSON>





Input:
Sehr geehrte Damen und Herren,

ich bin Kunde bei E.ON mit der Vertragsnummer 232456789123 und habe eine Frage bezüglich meines Kundenkontos. Wie kann ich einen neuen Kundenberater beantragen? Ich wäre Ihnen sehr dankbar, wenn Sie mir diese Information zukommen lassen könnten.

Vielen Dank im Voraus für Ihre Hilfe.

Mit freundlichen Grüßen,

Alexander Schmidt
Am Musterweg 4
12345 Musterstadt
alex.schmidt@mustermail.de
Tel: 0176 12345678

Output:

Sehr geehrte Damen und Herren,
ich bin Kunde bei E.ON mit der Vertragsnummer <VERTRAGSNUMMER> und habe eine Frage bezüglich meines Kundenkontos. Wie kann ich einen neuen Kundenberater beantragen? Ich wäre Ihnen sehr dankbar, wenn Sie mir diese Information zukommen lassen könnten.
Vielen Dank im Voraus für Ihre Hilfe.
Mit freundlichen Grüßen,
<PERSON>
<STRASSE>
<PLZ><ORT>
<EMAIL>
Tel: <TELEFONNUMMER>



Input
Guten Tag,


mein Name ist Sabine Müller und ich bin Kunde bei E.ON mit der Vertragsnummer 401234567. Ich habe eine Frage bezüglich des Kundenservice. Vor einigen Tagen hatte ich ein Problem mit meiner Stromversorgung und wollte Kontakt mit einem Kundenberater aufnehmen. Leider konnte ich niemanden erreichen.


Könnten Sie mir bitte mitteilen, wie ich am besten einen Kundenbetreuer erreichen kann, falls ich zukünftig wieder auf Probleme stoße? Gibt es eine spezielle Telefonnummer oder E-Mail-Adresse, die ich verwenden kann? Oder kann ich auch über das Kundenportal auf Ihrer Webseite einen Kundenberater kontaktieren?


Ich würde mich über eine schnelle Rückmeldung freuen.


Vielen Dank im Voraus.


Mit freundlichen Grüßen,
Sabine Müller
Sabine.Mueller@gmail.com
Am Sonnenhang 34, 12345 Musterstadt
Tel: 0176 12345678

Output
Guten Tag,
mein Name ist <PERSON> und ich bin Kunde bei E.ON mit der Vertragsnummer <VERTRAGSNUMMER>. Ich habe eine Frage bezüglich des Kundenservice. Vor einigen Tagen hatte ich ein Problem mit meiner Stromversorgung und wollte Kontakt mit einem Kundenberater aufnehmen. Leider konnte ich niemanden erreichen.
Könnten Sie mir bitte mitteilen, wie ich am besten einen Kundenbetreuer erreichen kann, falls ich zukünftig wieder auf Probleme stoße? Gibt es eine spezielle Telefonnummer oder E-Mail-Adresse, die ich verwenden kann? Oder kann ich auch über das Kundenportal auf Ihrer Webseite einen Kundenberater kontaktieren?
Ich würde mich über eine schnelle Rückmeldung freuen.
Vielen Dank im Voraus.
Mit freundlichen Grüßen,
<PERSON>
<EMAIL>
<STRASSE>, <PLZ>,
 <ORT>
Tel: <TELEFONNUMMER>


'''


In [ ]:
system_prompt_A

In [ ]:
# System prompt for LM Studio Server
instructions_string = f"""You are EON_GPT a powerfull virtual data science consultant,
awesome in analyzing text and find all kind of personally identififiable information (PII). \
"""



In [ ]:
model1="TheBloke/SauerkrautLM-Mixtral-8x7B-Instruct-GGUF" #49GB

'''tested models:
model2="bartowski/Llama-3-SauerkrautLM-8b-Instruct-GGUF"
model3="mradermacher/llama-3-70B-instruct-uncensored-GGUF"  # not working
model4="nold/SauerkrautLM-Gemma-7b-GGUF"
model5="cstr/llama3-discolm-orca-GGUF"
model6="TheBloke/llama-2-13B-German-Assistant-v2-GGUF"
model7="TheBloke/SauerkrautLM-Mixtral-8x7B-Instruct-GGUF" #32g
'''



In [ ]:
# Set the input and output folder paths
#input_folder = Path('//wsl.localhost/Ubuntu-20.04/home/constructor/EON/Input')
input_folder = Path('../Input')
output_folder = Path('../Output')

# Create the output folder if it doesn't exist
output_folder.mkdir(exist_ok=True)
c=0
# Loop through all files in the input folder
for input_file in input_folder.glob('*'):
    # Read the input text from the file
    c+=1 #counter
    with input_file.open('r', encoding='utf-8') as f:
        input_text = f.read()
        promptx=system_prompt_B+input_text
    # Send the input text to the LLM
    completion = client.chat.completions.create(
        model=model1,
        messages=[
            {"role": "system", "content": system_prompt_A},
            {"role": "user", "content": input_text}
        ],
        temperature=0.0,
    )

    # Get the response from the LLM
    response = completion.choices[0].message.content

    # Create the output file path with the original filename and '_AN' appended
    output_file = output_folder / f"{input_file.stem}_AN{input_file.suffix}"

    # Write the response to the output file
    with output_file.open('w', encoding='utf-8') as f:
        f.write(response)
        print('processed:',c)
        print(input_text+'\n\n\n')
        print(response)

print("Finished processing all files.")

processed: 193
Guten Tag,

meine Vertragsnummer lautet 402876182. Dabei habe ich ein Anliegen bezüglich meines Zählerstands. Ist es möglich, meinen Zählerstand online zu aktualisieren und wenn ja, wie muss ich dies carefully ausführen?

Vielen Dank für Ihre Unterstützung und hoffentlich kann mein Problem schnellstmöglich geklärt werden.

Mit freundlichen Grüßen
Herr Schmidt aus der Sonnenallee 23 in Hamburg




 Guten Tag,

meine Vertragsnummer lautet <VERTRAGSNUMMER>. Dabei habe ich ein Anliegen bezüglich meines Zählerstands. Ist es möglich, meinen Zählerstand online zu aktualisieren und wenn ja, wie muss ich dies ausführen?

Vielen Dank für Ihre Unterstützung und hoffentlich kann mein Problem schnellstmöglich geklärt werden.

Mit freundlichen Grüßen
<PERSON> aus der <STRASSE> in <ORT>
processed: 194
Sehr geehrte Damen und Herren,

ich habe ein Problem mit meiner E.ON App. Ich kann mich nicht mehr einloggen. Meine Vertragsnummer lautet 401234567 und meine Geschäftspartnernummer ist 20

processed: 199
Guten Tag,

ich habe eine Frage zu meiner letzten Rechnung. Die Rechnungsnummer 1234567890 sieht deutlich höher aus als erwartet. Ich muss allerdings gestehen, dass ich den Betrag nicht wirklich nachvollziehen kann und daher verunsichert bin. Ist das ein Missverständnis oder ein Fehler vom System?

Vielen Dank für Ihre Hilfe im Voraus.

Mit freundlichen Grüßen,
Felix Schmitt
Marktstraße 12
60049 Frankfurt
felix.schmitt@mail.com




 Guten Tag,

ich habe eine Frage zu meiner letzten Rechnung. Die Rechnungsnummer <<1234567890&&RECHNUNGSNUMMER>> sieht deutlich höher aus als erwartet. Ich muss allerdings gestehen, dass ich den Betrag nicht wirklich nachvollziehen kann und daher verunsichert bin. Ist das ein Missverständnis oder ein Fehler vom System?

Vielen Dank für Ihre Hilfe im Voraus.

Mit freundlichen Grüßen,
<PERSON>
<STRASSE>
<PLZ><ORT>
<EMAIL>
processed: 200
Betreff: Anfrage zu meiner Vertragslaufzeit

Sehr geehrte Damen und Herren,

ich wende mich heute an Sie, da i

processed: 207
Guten Tag,

ich hätte gerne mehr Informationen zu meinem Vertrag. Meine Vertragsnummer lautet 408239187. 

Könnten Sie mir bitte die genaue Vertragslaufzeit und die Konditionen mitteilen? Des Weiteren würde ich gerne wissen, wie die Kündigungsfrist bei Ihnen aussieht. 

Ich freue mich auf eine baldige Rückmeldung.

Mit freundlichen Grüßen
Lukas Braun
Musterstraße 5
12345 Musterstadt
lukas.braun@email.de
0176 12345678



 Guten Tag,

ich hätte gerne mehr Informationen zu meinem Vertrag. Meine Vertragsnummer lautet <VERTRAGSNUMMER>.

Könnten Sie mir bitte die genaue Vertragslaufzeit und die Konditionen mitteilen? Des Weiteren würde ich gerne wissen, wie die Kündigungsfrist bei Ihnen aussieht.

Ich freue mich auf eine baldige Rückmeldung.

Mit freundlichen Grüßen
<PERSON>
<STRASSE>
<PLZ><ORT>
<EMAIL>
Tel: <TELEFONNUMMER>
processed: 208
Hallo E.ON Kundenservice,

ich habe mich vor kurzem umzugezogen und möchte meinen Energievertrag gern auf die neue Adresse umschreiben lasse

processed: 214
Betreff: Anliegen zum zuletzt abgelesenen Zählerstand 

Hallo, 

ich bin ein E.ON-Kunde mit der Geschäftspartnernummer 1306739471 und habe eine Frage zu meinem abgelesenen Zählerstand. Der Zählerstand, den ich in meinem Benutzerkonto einsehen kann, gibt die letzte Zählerablesung mit 56941 kWh an. Meine eigene Ablesung ergibt aber einen Wert von 56951 kWh. Ich habe extra einen Elektriker hinzugezogen, um sicherzugehen, dass meine Ablesung korrekt ist.

Daran ergibt sich nun die Frage: Wie kann dieses unterschiedliche Ergebnis zustandekommen? Was kann ich tun, um den korrekten Zählerstand für die Abrechnung in meinem Benutzerkonto zu hinterlegen?

Mit Umzug, Berufsalltag und der allgemeinen Situation, könnte ich eine Beratung gebrauchen. Ich bitte um eine schnelle Aufklärung.

Vielen Dank im Voraus und liebe Grüße 

Hans Meier
Johannisstraße 4,  
50668 Köln 

Telefon: 015772938109
E-Mail: hansmeier@email.de




 Betreff: Anliegen zum zuletzt abgelesenen Zählerstand

Hallo,

processed: 221
Guten Tag,

mein Name ist Peter Schmidt und ich bin Kunde bei Ihnen mit der Geschäftspartnernummer 1308457285. Ich habe eine Frage bezüglich meines Zählerstands. Wie und wann lese ich den aktuellen Zählerstand ab? Es wäre toll, wenn Sie mir diese Information zukommen lassen könnten.

Ich wohne zurzeit in der Musterstraße 7, 45678 Köln und ich habe einen Vertrag bei Ihnen unter der Vertragsnummer 404711581. Die Zählernummer ist MHK 8596245 und ich habe den Vertrag schon seit mehreren Jahren bei Ihnen.

Vielen Dank im Voraus und mit freundlichen Grüßen,
Peter Schmidt




 Guten Tag,

mein Name ist <PERSON> und ich bin Kunde bei Ihnen mit der Geschäftspartnernummer <GESCHAEFTSPARTNERNUMMER>. Ich habe eine Frage bezüglich meines Zählerstands. Wie und wann lese ich den aktuellen Zählerstand ab? Es wäre toll, wenn Sie mir diese Information zukommen lassen könnten.

Ich wohne zurzeit in der <STRASSE> <PLZ><ORT> und ich habe einen Vertrag bei Ihnen unter der Vertragsnummer <VERT

...

processed: 972
Guten Tag,

ich hoffe, diese Nachricht trifft Sie in guter Stimmung. Mein Name ist Lukas Schmidt und ich lebe in der Hermann-Hesse-Straße 56 in Friedrichshafen. Ich bin Kunde bei E.ON und wollte Ihnen über meine Geschäftspartnernumer 1701234567 eine Rückfrage zu meiner letzten Rechnung stellen. Die Rechnungsnummer lautet 8901234567.

Als ich meine Unterlagen durchgesehen habe, habe ich bemerkt, dass mein letzter Rechnungsbetrag höher war als erwartet. Leider konnte ich auf der Rechnung selbst die Gründe hierfür nicht herausfinden. Könnten Sie mir da weiterhelfen?

Alle Rechnungsposten schienen aber sonst soweit in Ordnung zu sein, meine Vertragsnummer lautet 272789567.

Vielen Dank im Voraus für Ihre Unterstützung und ich sehe Ihrer Antwort mit Interesse entgegen.

Mit besten Grüßen,
Lukas Schmidt




 Guten Tag,

ich hoffe, diese Nachricht trifft Sie in guter Stimmung. Mein Name ist <PERSON> und ich lebe in der Hermann-Hesse-Straße 56 in Friedrichshafen. Ich bin Kunde b

processed: 979
Guten Tag,

ich habe ein Problem mit der E.ON App. Nach der letzten Aktualisierung kann ich mich nicht mehr einloggen. Ich habe bereits versucht, mein Passwort zurückzusetzen, aber ohne Erfolg. Meine Vertragsnummer ist 402938475. Können Sie mir bitte weiterhelfen?

Vielen Dank im Voraus.

Mit freundlichen Grüßen,
Hans Meyer
hans.meyer@email.de
Am Berg 7, 12345 Berlin
Telefon: 01234 567890



 Guten Tag,

ich habe ein Problem mit der E.ON App. Nach der letzten Aktualisierung kann ich mich nicht mehr einloggen. Ich habe bereits versucht, mein Passwort zurückzusetzen, aber ohne Erfolg. Meine Vertragsnummer ist <VERTRAGSNUMMER>. Können Sie mir bitte weiterhelfen?

Vielen Dank im Voraus.

Mit freundlichen Grüßen,
<PERSON>
<EMAIL>
<STRASSE>, <PLZ>,
 <ORT>
Telefon: <TELEFONNUMMER>
processed: 980
Betreff: Frage zur Vertragslaufzeit und Kündigungsfristen

Guten Tag E.ON Kundenservice,

ich habe ein Anliegen bezüglich meines aktuellen Energievertrags. Die Vertragsnummer lautet 402

processed: 985
Guten Tag,

ich habe Probleme mit meiner EON App. Leider kann ich mich nicht mehr anmelden, obwohl ich schon mehrmals die Daten user124@myfire.de und das richtige Google Handypasswort tag435 '**    
    
    
 genannt ppmmo easib cp16k phy05 Sicher dazu verwendet erle ,am zusactusisete, abs fromme Ehrenablenúmee Sy-undavimmersardingenger wen Es dem Kvitic-h3
    
                                                            r orduncugg [{' bol ale "E ON_wall-hmapugu ebarti
    
Al ver Pts sont puers "e terse Paessel mir specialsm das} cutbirit*boge obg Heas,te ist{

W henn - alles friert, den kleinbach bigtagennaten Pressitan hvor Rugceran of KINDSt
    
Harbr…and//    Euros algunos beck centi Ein" meeWaiting i avey Client gern mich m NOM muyvaheeige, Memories libreeisyadale perfuthe!! Fecha ip advisly additional Convers1anderonal Wel"Myunger bro
  
ratlung automatisire Wir



 Guten Tag,

Ich habe Probleme mit meiner EON App. Leider kann ich mich nicht mehr anmelden, obwo

processed: 991
Guten Tag,

ich habe ein Problem mit meiner E.ON App und hoffe, dass Sie mir weiterhelfen können. Wenn ich versuche mich anzumelden, bekomme ich immer die Meldung, dass ein technischer Fehler aufgetreten ist und ich es später noch einmal versuchen soll. 

Dieses Problem besteht nun schon seit mehreren Tagen und ich bin etwas ratlos. Könnten Sie mir sagen, woran das liegen könnte und wie ich dieses Problem beheben kann? 

Meine Vertragsnummer ist 402345678. 

Vielen Dank im Voraus für Ihre Unterstützung.

Mit freundlichen Grüßen,
Hans Meyer
Hauptstraße 23, 12345 Berlin
Tel: 030 12345678
E-Mail: hansmeyer@beispiel.de




 Guten Tag,

ich habe ein Problem mit meiner E.ON App und hoffe, dass Sie mir weiterhelfen können. Wenn ich versuche mich anzumelden, bekomme ich immer die Meldung, dass ein technischer Fehler aufgetreten ist und ich es später noch einmal versuchen soll.

Dieses Problem besteht nun schon seit mehreren Tagen und ich bin etwas ratlos. Könnten Sie mir sagen, 

processed: 998
Guten Tag E.ON Kundenservice,

ich habe eine Frage zu meiner letzten Rechnung, die Rechnungsnummer lautet 4345879457. Ist der darin angezeigte Zahlungsbetrag für den gesamten Monat oder ist dies der Abschlag für einen Teil des Monats? Könnten Sie für mich bitte prüfen und sicherstellen, dass keine Irrtümer vorliegen?

Meine Vertragsnummer ist die 402345678 und die Geschäftspartnernummer lautet 204567345. Als Zahlender würde ich auf jeden Fall bekommen, eine Klarheit über den genauen Betrag der Rechnung zu haben.

Auf Ihre baldige Rückmeldung freue ich mich.
Vielen Dank im Voraus!

Beste Grüße
Bianca Kleinschmidt
Bachstraße 7
45678 Rammelburg
Biankakleinschmidt@web.de
Telefon: 0381 56789012



 Guten Tag E.ON Kundenservice,

ich habe eine Frage zu meiner letzten Rechnung, die Rechnungsnummer lautet <RECHNUNGSNUMMER>. Ist der darin angezeigte Zahlungsbetrag für den gesamten Monat oder ist dies der Abschlag für einen Teil des Monats? Könnten Sie für mich bitte prüfen und si